In [14]:
using ModelingToolkit, OrdinaryDiffEq

# 定义变量
@variables t
@variables V₁(t) V₂(t) V₃(t) I(t)
# @variables V₁(t) [irreducible = true] V₂(t) [irreducible = true] V₃(t) [irreducible = true] I(t) [irreducible = true]

# 定义微分
D = Differential(t)

# 设置参数
R = 1.0
C = 1.0
V = 1.0

# 输入方程
rc_eqs = [
    V₁ - V₃ ~ V
    V₁ - V₂ ~ I * R
    D(V₂) ~ I / C
    V₃ ~ 0
]

# 构建系统
@named rc_model = ODESystem(rc_eqs, t)
rc_model |> display
rc_model |> states |> display
# 系统化简
sys = structural_simplify(rc_model)

Model rc_model with 4 equations
States (4):
  V₂(t)
  V₃(t)
  V₁(t)
  I(t)
Parameters (0):

4-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 V₂(t)
 V₃(t)
 V₁(t)
 I(t)

Model rc_model with 1 equations
States (1):
  V₂(t)
Parameters (0):
Incidence matrix:1×2 SparseArrays.SparseMatrixCSC{Num, Int64} with 2 stored entries:
 ×  ×

In [22]:
sys |> println

ODESystem(0x0000000000000002, Equation[Differential(t)(V₂(t)) ~ I(t)], t, Any[V₂(t)], Any[], nothing, Dict{Any, Any}(:I => I(t), :V₂ => V₂(t), :V₃ => V₃(t), :V₁ => V₁(t)), Any[], Equation[V₃(t) ~ 0.0, V₁(t) ~ 1.0 + V₃(t), I(t) ~ V₁(t) - V₂(t)], Base.RefValue{Vector{Num}}(Num[]), Base.RefValue{Any}(Matrix{Num}(undef, 0, 0)), Base.RefValue{Any}(Matrix{Num}(undef, 0, 0)), Base.RefValue{Matrix{Num}}(Matrix{Num}(undef, 

0, 0)), Base.RefValue{Matrix{Num}}(Matrix{Num}(undef, 0, 0)), :rc_model, ODESystem[], Dict{Any, Any}(), nothing, nothing, nothing, ModelingToolkit.SymbolicContinuousCallback[ModelingToolkit.SymbolicContinuousCallback(Equation[], Equation[])], ModelingToolkit.SymbolicDiscreteCallback[], nothing, nothing, TearingState of ODESystem, ModelingToolkit.Substitutions(Equation[V₃(t) ~ 0.0, V₁(t) ~ 1.0 + V₃(t), I(t) ~ V₁(t) - V₂(t)], [Int64[], [1], [1, 2]], nothing), true, nothing, nothing, nothing, ODESystem(0x0000000000000002, Equation[Differential(t)(V₂(t)) ~ I(t), V₁(t) - V₃(t) ~ 1.0, V₁(t) - V₂(t) ~ I(t), V₃(t) ~ 0], t, SymbolicUtils.BasicSymbolic{Real}[V₂(t), V₃(t), V₁(t), I(t)], Any[], nothing, Dict{Any, Any}(:I => I(t), :V₂ => V₂(t), :V₃ => V₃(t), :V₁ => V₁(t)), Any[], Equation[], Base.RefValue{Vector{Num}}(Num[]), Base.RefValue{Any}(Matrix{Num}(undef, 0, 0)), Base.RefValue{Any}(Matrix{Num}(undef, 0, 0)), Base.RefValue{Matrix{Num}}(Matrix{Num}(undef, 0, 0)), Base.RefValue{Matrix{Num}}(Ma

In [23]:
sys |> states |> display

1-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 V₂(t)

In [24]:
# 设置初值
u0 = [V₂ => 0.0]
# 求解时间范围
tspan = (0.0, 10.0)
# 构建问题并求解
prob = ODAEProblem(sys, u0, tspan)
sol = solve(prob, Tsit5());

In [26]:
# 查看V₂的变化
sol[V₂]

19-element Vector{Float64}:
 0.0
 9.999500016666247e-5
 0.001099395221772342
 0.011038622307372232
 0.07387132069817183
 0.20198030129465658
 0.35954472556518147
 0.5167641452122544
 0.6637713825376198
 0.7841482271226826
 0.8737784044472122
 0.9331779821204208
 0.9684489616227439
 0.9869310637824636
 0.9953772824474669
 0.9986536797747498
 0.9996930416967437
 0.9999471971800106
 0.9999499280997356

In [1]:
using ModelingToolkit, OrdinaryDiffEq
# 定义独立时间变量
@variables t
# 器件端口作为连接点
@connector function Pin(; name)
    sts = @variables v(t) = 1.0 i(t) = 1.0 [connect = Flow]
    return ODESystem(Equation[], t, sts, []; name=name)
end
# 地，地的端口名字改为g，不是port
function Ground(; name)
    @named g = Pin()
    eqs = [g.v ~ 0; g.i ~ 0]
    return compose(ODESystem(eqs, t, [], []; name=name), g)
end
# 电阻元件
function Resistor(; name, R=1.0)
    @named p = Pin()
    @named n = Pin()
    ps = @parameters R = R
    eqs = [
        p.v - n.v ~ p.i * R
        0 ~ p.i + n.i
    ]
    return compose(ODESystem(eqs, t, [], ps; name=name), p, n)
end
# 电容元件，因为不能对表达式进行微分，所以再引入一个电容的电压差变量v
function Capacitor(; name, C=1.0)
    @named p = Pin()
    @named n = Pin()
    ps = @parameters C = C
    sts = @variables v(t) = 1.0
    D = Differential(t)
    eqs = [
        v ~ p.v - n.v
        D(v) ~ p.i / C
        0 ~ p.i + n.i
    ]
    return compose(ODESystem(eqs, t, sts, ps; name=name), p, n)
end
# 电压源
function ConstantVoltage(; name, V=1.0)
    @named p = Pin()
    @named n = Pin()
    ps = @parameters V = V
    eqs = [
        V ~ p.v - n.v
        0 ~ p.i + n.i
    ]
    return compose(ODESystem(eqs, t, [], ps; name=name), p, n)
end
# 定义组件
R = 1.0
C = 1.0
V = 1.0
@named resistor = Resistor(R=R) # @name可以视为给定义的组件起了个名字，Resistor(R=R)返回的组件名字就叫resistor
@named capacitor = Capacitor(C=C)
@named source = ConstantVoltage(V=V)
@named ground = Ground()
# 构建连接关系
rc_eqs = [
    connect(source.p, resistor.p)
    connect(resistor.n, capacitor.p)
    connect(capacitor.n, source.n, ground.g)
]
@named _rc_model = ODESystem(rc_eqs, t) #连接关系也需要放到ODESystem中。
# 组件与组件连接关系一起构建系统
@named rc_model = compose(_rc_model, [resistor, capacitor, source, ground])
equations(rc_model) # 查看方程
# 系统化简
sys = structural_simplify(rc_model)
equations(sys) # 查看方程
# 定义初值
u0 = [capacitor.v => 0.0]
# 求解
prob = ODAEProblem(sys, u0, (0, 10.0))
sol = solve(prob, Tsit5())
# 查看电容电压变化
sol[capacitor.p.v]

19-element Vector{Float64}:
 0.0
 9.999500016666247e-5
 0.001099395221772342
 0.011038622307372232
 0.07387132069817183
 0.20198030129465658
 0.35954472556518147
 0.5167641452122544
 0.6637713825376198
 0.7841482271226826
 0.8737784044472122
 0.9331779821204208
 0.9684489616227439
 0.9869310637824636
 0.9953772824474669
 0.9986536797747498
 0.9996930416967437
 0.9999471971800106
 0.9999499280997356

In [7]:
sys.resistor.p.v

Num

In [1]:
# 最新版MTK
using ModelingToolkit, OrdinaryDiffEq
@variables t
@connector Pin begin
    v(t)
    i(t), [connect = Flow]
end
@mtkmodel Ground begin
    @components begin
        g = Pin()
    end
    @equations begin
        g.v ~ 0
    end
end
@mtkmodel OnePort begin
    @components begin
        p = Pin()
        n = Pin()
    end
    @variables begin
        v(t)
        i(t)
    end
    @equations begin
        v ~ p.v - n.v
        0 ~ p.i + n.i
        i ~ p.i
    end
end
@mtkmodel Resistor begin
    @extend OnePort()
    @parameters begin
        R = 1.0 # Sets the default resistance
    end
    @equations begin
        v ~ i * R
    end
end

D = Differential(t)

@mtkmodel Capacitor begin
    @extend OnePort()
    @parameters begin
        C = 1.0
    end
    @equations begin
        D(v) ~ i / C
    end
end

@mtkmodel ConstantVoltage begin
    @extend OnePort()
    @parameters begin
        V = 1.0
    end
    @equations begin
        V ~ v
    end
end

@mtkmodel RCModel begin
    @components begin
        resistor = Resistor(R=1.0)
        capacitor = Capacitor(C=1.0)
        source = ConstantVoltage(V=1.0)
        ground = Ground()
    end
    @equations begin
        connect(source.p, resistor.p)
        connect(resistor.n, capacitor.p)
        connect(capacitor.n, source.n, ground.g)
    end
end

@mtkbuild rc_model = RCModel(resistor.R=2.0)

u0 = [
    rc_model.capacitor.v => 0.0
]
prob = ODEProblem(rc_model, u0, (0, 10.0))
sol = solve(prob, Tsit5())
sol[rc_model.capacitor.p.v]

15-element Vector{Float64}:
 0.0
 4.999875002083307e-5
 0.000549848777725354
 0.005534627202823254
 0.05304729754059866
 0.16870857122402544
 0.3232210676188638
 0.47918255576870894
 0.6305653418370613
 0.7574167496079482
 0.854764329162927
 0.9210274311861326
 0.9615821238825505
 0.9835167746379931
 0.9932587425735419

In [8]:
rc_model.capacitor

Model capacitor with 4 (6) equations
States (6):
  v(t)
  i(t)
  p₊v(t)
  p₊i(t)
⋮
Parameters (1):
  C [defaults to 1.0]

In [9]:
rc_model.capacitor.v

capacitor₊v(t)